# CDSDS 542 Deep Learning for Data Science - Discussion 8: Transformer

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/DL4DS/fa2026_discussions/blob/main/discussion_8.ipynb)

**Module**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import math
import random
import collections
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split as tts

Main translator structure before transformer:
![image](https://eugeneyan.com/assets/encoder-decoder.jpeg)

## Demo 1: Sequence Reversal with Transformer

Goal:
- Input: sequence of integers [1, 2, 3, 4]
- Target: reversed sequence [4, 3, 2, 1]

![img](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*kf871smtQKXAf3dSYOlVPA.png)

Two main parts to a transformer:

- Encoder:  The encoder takes in the input sentence and produces a fixed-size vector representation of it
- Decoder: The decoder takes the fixed-size vector representation and produces the output sentence. The decoder uses both self-attention and cross-attention, where the attention mechanism is applied to the output of the encoder and the input of the decoder.

In [2]:
# create the dataset

# data = [
#     ([1, 2, 3, 4], [4, 3, 2, 1]),
#     ([5, 6, 7, 8], [8, 7, 6, 5]),
#     ([7, 8, 9, 10], [10, 9, 8, 7]),
#     ([2, 3, 4, 5], [5, 4, 3, 2])
# ]

data = []
for _ in range(500):
    length = random.randint(3, 10)
    seq = list(range(1, length+1))
    data.append((seq, seq[::-1]))

for _ in range(500):
    start = random.randint(1, 100)
    seq = list(range(start, start+5))
    data.append((seq, seq[::-1]))


pad = 0
vocab_size = 500
max_len = max(len(x[0]) for x in data)

def pad_seq(seq):
    return seq + [pad] * (max_len - len(seq))

inputs = torch.tensor([pad_seq(x[0]) for x in data])
targets = torch.tensor([pad_seq(x[1]) for x in data])

# print("Input:\n", inputs)
# print("Target:\n", targets)

### 1. `Embedding`:

`nn.Embedding(vocab_size, d_model, padding_idx)`

This step is for word vector mapping. Input a sequence of tokens and output the vector representation of each token, converting discrete ids into computable vectors

- `vocab_size`: Vocabulary list size, +1 to allocate an additional position for the padding token.
- `d_model`: The Embedding dimension (how many dimensional vectors are used to represent each token), determines the number of columns in the Embedding matrix.
- `padding_idx`: Specify which token ID is used for padding, the embedding at this position has a gradient of 0 during training.

Why Padding?
- Neural networks usually require batch processing to improve efficiency. However, different sequences have different lengths and cannot directly form a matrix.

In [ ]:
vocab_size = 20
d_model = 4

word_embeddings = nn.Embedding(vocab_size+1, d_model, padding_idx=0)
embedded = word_embeddings(torch.tensor([1, 2, 3, 4]))
print("Embedded input: ", embedded)

Embedded input:  tensor([[ 0.3649, -0.0367,  0.4655, -0.7512],
        [-0.0338, -2.1567, -0.6836, -0.9363],
        [-1.3292,  0.5497, -0.6537, -0.4422],
        [-0.4421,  0.0802,  0.1117,  0.4469]], grad_fn=<EmbeddingBackward0>)


### 2. `PositionalEncoding` class:
- Adds positional information to the input embeddings by summing them with positional encodings of the same dimensions. The positional encodings are calculated using sine and cosine functions of different frequencies.

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=50):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)


In [ ]:
pos_encoding = PositionalEncoding(d_model=4)
pe_matrix = pos_encoding.pe[0, : 4, :].detach().numpy()
pe_matrix

array([[ 0.        ,  1.        ,  0.        ,  1.        ],
       [ 0.84147096,  0.54030234,  0.00999983,  0.99995   ],
       [ 0.9092974 , -0.41614684,  0.01999867,  0.9998    ],
       [ 0.14112   , -0.9899925 ,  0.0299955 ,  0.99955004]],
      dtype=float32)

So, for this situation:

- embedded (4×4) : Random word vectors representing "what word it is"
- pe_matrix (4×4) : Fixed position encoding, representing "at which position"
- output = embedded + pe_matrix: Knowing both "what word" and "where"

### 3. `DotProductAttention` class:

Computes the attention scores by performing a dot product between queries (Q) and keys (K), scales these scores by the square root of the query dimension, applies a mask based on valid lengths to ignore padding, applies softmax to get probabilities, and finally computes the weighted sum of values (V).

- A “query” is analogous to a search query in a database. It represents the current item (e.g., a word or token in a sentence) the model focuses on or tries to understand. The query is used to probe the other parts of the input sequence to determine how much attention to pay to them.
- The “key” is like a database key used for indexing and searching. In the attention mechanism, each item in the input sequence (e.g., each word in a sentence) has an associated key. These keys are used to match with the query.
- The “value” in this context is similar to the value in a key-value pair in a database. It represents the actual content or representation of the input items. Once the model determines which keys (and thus which parts of the input) are most relevant to the query (the current focus item), it retrieves the corresponding values.

How to understand Q, K, V:

| Symbol | Mean     | Analogy      |
| -- | ------ | ------- |
| Q  | What am I looking for?| The person asking the question.|
| K  | What do I have to offer? | Information index tags.|
| V  | What information should I share if chosen? | The content of my answer itself |

- From basic linear-algebra, we know that matrices are nothing but the linear transformations or rules that operate on vectors and change their properties like rotate them by a certain angle, reflect them about some axis, etc
- These `trainable matrices` for query, keys and values do something similar – stretch, shear, or elongate the manifolds such that the `similarity of the alike words increases whereas for dissimilar words it decreases.`

- In a nutshell, transforming vectors with matrices can increase/decrease the similarity score and hence the attention weights between two vectors.
    - This is what K, Q and V do to the input embedding vectors. They are trainable meaning during the course of training, their weights will be optimized to change the manifold. This will increase/decrease the similarity between tokens on the basis of the loss function optimization during training.

![img](https://storrs.io/content/images/2021/08/Screen-Shot-2021-08-07-at-7.51.37-AM.png)

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def forward(self, Q, K, V, mask=None):
        # Q,K,V shapes: (batch, heads, seq, d_k)
        scores = Q @ K.transpose(-2, -1) / math.sqrt(Q.size(-1))  # QK^T / sqrt(d_k)
        if mask is not None:
            if mask.dim() == 2:
                mask = mask.unsqueeze(0).unsqueeze(0)  # (1,1,L,L)

            mask = mask.to(dtype=torch.bool, device=scores.device)
            scores = scores.masked_fill(mask, float('-inf'))
        attn = scores.softmax(dim=-1)
        return attn @ V, attn

In [ ]:
# calculate with an example
batch, heads, seq_len, d_k = 1, 1, 4, 8
torch.manual_seed(42)
Q = torch.randn(batch, heads, seq_len, d_k) * 0.5
K = torch.randn(batch, heads, seq_len, d_k) * 0.5
V = torch.randn(batch, heads, seq_len, d_k) * 0.5

# using Attention
attention = ScaledDotProductAttention()
output, attn_weights = attention(Q, K, V)

### Exercise 1

In [ ]:
batch, heads, seq_len, d_k = 1, 1, 4, 8
torch.manual_seed(42)
Q = torch.randn(batch, heads, seq_len, d_k) * 0.5
K = torch.randn(batch, heads, seq_len, d_k) * 0.5
V = torch.randn(batch, heads, seq_len, d_k) * 0.5

# TODO: calculate the attention block by hand
K_T = K.transpose(-2, -1)
scores = __fill__
print(f"scores: [1,1,4,8] @ [1,1,8,4] = {scores.shape}")
attn = __fill__
attn_np = attn[0, 0].detach().numpy() # attention weight matrix
print(f"sum of rows = {attn_np.sum(axis=1)}")
output =  __fill__  # [1,1,4,8]
print(f"attn @ V: [1,1,4,4] @ [1,1,4,8] = {output.shape}")

In [ ]:
# expected output

scores: [1,1,4,8] @ [1,1,8,4] = torch.Size([1, 1, 4, 4])
sum of rows = [1. 1. 1. 1.]
attn @ V: [1,1,4,4] @ [1,1,4,8] = torch.Size([1, 1, 4, 8])


In [ ]:
# TODO: check if your answer is correct
torch.allclose(torch.tensor(attn_np), attn_weights)

True

#### Attention Mask

The `mask` parameter controls which positions can attend to which positions. When `mask[i,j] == 0`, the attention mechanism prevents position `i` from attending to position `j`.

**Mechanism:**
```python
scores = scores.masked_fill(mask == 0, -1e9)
```
Positions where `mask == 0` are filled with `-1e9` (negative infinity). After `softmax()`, these positions get attention weights ≈ 0, effectively "hiding" them from the model.

**Two Common Types:**

1. **Padding Mask** - Prevents attention to padding tokens (`<PAD>`)
   - Shape: `[batch, 1, 1, seq_len]` or `[batch, 1, seq_len, seq_len]`
   - Example: `[1, 1, 1, 0]` masks the last position
   - Purpose: Ignore meaningless padding positions in variable-length sequences

2. **Causal Mask** - Prevents attention to future positions (autoregressive models)
   - Shape: Lower triangular matrix
      ```
        [[1, 0, 0, 0],    # token 0 can only see token 0
          [1, 1, 0, 0],    # token 1 can see tokens 0-1
          [1, 1, 1, 0],    # token 2 can see tokens 0-2
          [1, 1, 1, 1]]    # token 3 can see tokens 0-3
      ```
   - Purpose: Used in GPT-style models to prevent "looking ahead" during training

### 4. `MultiHeadAttention` class:

Transforms the input queries, keys, and values using the initialized weight matrices, splits these transformations into multiple heads, applies dot-product attention in each head, concatenates the heads' outputs, and finally linearly transforms the concatenated output.  

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        # Q,K,V trans
        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)

        self.attention = ScaledDotProductAttention()
        self.out = nn.Linear(d_model, d_model)

    def forward(self, x_q, x_k, x_v, mask=None):
        B, Lq, D = x_q.size()
        B, Lk, D = x_k.size()
        B, Lv, D = x_v.size()

        # (B, L, D) → (B, num_heads, L, d_k)
        Q = self.W_Q(x_q).contiguous().view(B, Lq, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_K(x_k).contiguous().view(B, Lk, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_V(x_v).contiguous().view(B, Lv, self.num_heads, self.d_k).transpose(1, 2)

        out, attn = self.attention(Q, K, V, mask)
        out = out.transpose(1,2).contiguous().view(B, Lq, D)
        return self.out(out), attn


**Scaled Dot-Product Attention**

Self-attention computes a weighted sum of values based on the similarity between queries and keys. The mechanism consists of three steps:

**Step 1: Compute Attention Scores**
```
scores = Q @ K^T / √d_k
```
- `Q @ K^T`: Compute pairwise similarity between all tokens → shape `[seq_len, seq_len]`
- `√d_k`: Scale factor to prevent gradients from vanishing when `d_k` is large
- `scores[i,j]`: Raw attention score representing how much token `i` should attend to token `j`

**Step 2: Normalize to Attention Weights**
```
attn = softmax(scores, dim=-1)
```
- Apply `softmax` row-wise to convert scores into probability distributions
- `attn[i,j]`: Normalized attention weight indicating the importance of token `j` when processing token `i`
- Each row sums to 1: `Σⱼ attn[i,j] = 1`

**Step 3: Compute Weighted Output**
```
output = attn @ V
```
- Aggregate value vectors using attention weights
- `output[i] = Σⱼ attn[i,j] × V[j]`: Each output is a weighted combination of all value vectors
- The model "gathers" information from relevant positions based on learned attention patterns

**Summary:** `Attention(Q, K, V) = softmax(QK^T / √d_k) × V`

### 5. `Encoder` Block


In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.attn = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(nn.Linear(d_model, d_ff), nn.ReLU(), nn.Linear(d_ff, d_model))
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(x + attn_out)
        x = self.norm2(x + self.ff(x))
        return x

The encoder block consists of two sub-layers, each followed by residual connection and layer normalization.

**Architecture:**
```python
x → Multi-Head Attention → Add & Norm → Feed-Forward → Add & Norm → output
    ↓_________________↑                   ↓___________↑
       residual connection               residual connection
```

**Components:**

1. **Multi-Head Attention**
   ```python
   attn_out, _ = self.attn(x, x, x)  # Self-attention: Q=K=V=x
   ```
   - Enables the model to attend to different positions and capture various relationships
   - Output shape: `[batch, seq_len, d_model]`

2. **First Add & Norm**
   ```python
   x = self.norm1(x + attn_out)
   ```
   - `x + attn_out`: Residual connection preserves original information and facilitates gradient flow
   - `norm1`: Layer normalization stabilizes training by normalizing across the feature dimension

3. **Feed-Forward Network (FFN)**
   ```python
   self.ff = nn.Sequential(
       nn.Linear(d_model, d_ff),  # Expand: d_model → d_ff (typically d_ff = 4 × d_model)
       nn.ReLU(),                  # Non-linearity
       nn.Linear(d_ff, d_model)   # Project back: d_ff → d_model
   )
   ```
   - Processes each position independently (position-wise)
   - Adds non-linear transformation capacity to the model

4. **Second Add & Norm**
   ```python
   x = self.norm2(x + self.ff(x))
   ```
   - Another residual connection followed by layer normalization

### 6. `Decoder` Block

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.ff = nn.Sequential(nn.Linear(d_model, d_ff), nn.ReLU(), nn.Linear(d_ff, d_model))
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

    def forward(self, x, enc_out):
        seq_len = x.size(1)
        mask = torch.triu(torch.ones(seq_len, seq_len), 1).bool().to(x.device)

        out, _ = self.self_attn(x, x, x, mask=mask)
        x = self.norm1(x + out)

        out, _ = self.cross_attn(x, enc_out, enc_out)
        x = self.norm2(x + out)

        x = self.norm3(x + self.ff(x))
        return x

The decoder block extends the encoder block with an additional cross-attention layer, enabling it to attend to the encoder's output. It consists of three sub-layers, each with residual connection and layer normalization.

**Architecture:**
```python
x → Masked Self-Attention → Add & Norm → Cross-Attention → Add & Norm → FFN → Add & Norm → output
    ↓___________________↑                 ↓___________↑              ↓_______↑
      residual connection                residual connection      residual connection
```

**Components:**

1. **Masked Self-Attention**
   ```python
   mask = torch.triu(torch.ones(seq_len, seq_len), 1).bool()  # Upper triangular
   out, _ = self.self_attn(x, x, x, mask=mask)
   x = self.norm1(x + out)
   ```
2. **Cross-Attention** (Encoder-Decoder Attention)
   ```python
   out, _ = self.cross_attn(x, enc_out, enc_out)  # Q=decoder, K=V=encoder
   x = self.norm2(x + out)
   ```
   - **Query (Q)**: From decoder (current decoding state)
   - **Key (K) & Value (V)**: From encoder output (source sequence information)
   - Allows decoder to attend to relevant parts of the input sequence

3. **Feed-Forward Network**
   ```python
   x = self.norm3(x + self.ff(x))
   ```
   - Same structure as encoder: `Linear(d_model → d_ff) → ReLU → Linear(d_ff → d_model)`
   - Position-wise transformation applied independently to each token



### 7. Define `Transformer`

In [ ]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model=32, num_heads=1, d_ff=64, num_layers=1):
        super().__init__()
        self.embed = nn.Embedding(vocab_size+1, d_model, padding_idx=0)
        self.pos = PositionalEncoding(d_model)

        self.encoder_layers = nn.ModuleList([
            EncoderBlock(d_model, num_heads, d_ff) for _ in range(num_layers)
        ])
        self.decoder_layers = nn.ModuleList([
            DecoderBlock(d_model, num_heads, d_ff) for _ in range(num_layers)
        ])

        self.fc = nn.Linear(d_model, vocab_size+1)

    def forward(self, src, tgt):
        # Encoder
        src = self.pos(self.embed(src))
        for layer in self.encoder_layers:
            src = layer(src)
        enc_out = src

        # Decoder
        tgt = self.pos(self.embed(tgt))
        for layer in self.decoder_layers:
            tgt = layer(tgt, enc_out)

        return self.fc(tgt)


The full Transformer architecture combines the encoder and decoder stacks with embedding and output layers.

**Architecture Overview:**
```python
Input (token IDs)
  ↓
Embedding + Positional Encoding
  ↓
Encoder Stack (N layers)
  ↓
Decoder Stack (N layers) ← receives encoder output
  ↓
Linear Projection (to vocabulary)
  ↓
Output (logits over vocabulary)
```

In [ ]:
vocab_size = 501
model = Transformer(vocab_size)
criterion = nn.CrossEntropyLoss(ignore_index=pad)
opt = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(300):
    opt.zero_grad()
    tgt_in = torch.cat([torch.full((targets.size(0), 1), 1, device=targets.device), targets[:, :-1]], dim=1)
    out = model(inputs, tgt_in)
    loss = criterion(out.view(-1, out.size(-1)), targets.view(-1))
    loss.backward()
    opt.step()
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss={loss.item():.4f}")

Epoch 0, Loss=6.1965
Epoch 50, Loss=0.0718
Epoch 100, Loss=0.0055
Epoch 150, Loss=0.0023
Epoch 200, Loss=0.0014
Epoch 250, Loss=0.0010


In [ ]:
def generate(model, src, max_len):
    model.eval()
    src = src.unsqueeze(0)

    enc = model.pos(model.embed(src))
    for layer in model.encoder_layers:
        enc = layer(enc)

    BOS = 1
    tgt = torch.tensor([[BOS]], device=src.device)

    for _ in range(max_len):
        dec = model.pos(model.embed(tgt))
        for layer in model.decoder_layers:
            dec = layer(dec, enc)
        logits = model.fc(dec)
        next_tok = logits[:, -1].argmax(-1, keepdim=True)
        tgt = torch.cat([tgt, next_tok], dim=1)

    return tgt[:, 1:]

test = torch.tensor([25,26,27,28])
print(generate(model, test, max_len=4))

tensor([[28, 27, 26, 25]])


## Demo 2: String Reversal

In [ ]:
sent_pairs = [
    ("I love deep learning", "learning deep love I"),
    ("Transformers are so powerful", "powerful so are Transformers"),
    ("Attention is a magic", "magic a is Attention"),
    ("Neural networks learn patterns", "patterns learn networks Neural"),
]

# build vocab
vocab = {"<pad>":0}
for s, t in sent_pairs:
    for w in (s + " " + t).split():
        if w not in vocab:
            vocab[w] = len(vocab)

inv_vocab = {i:w for w,i in vocab.items()}
pad = vocab["<pad>"]
vocab_size = len(vocab)

max_len = max(len(s.split()) for s,_ in sent_pairs)

def pad_seq(seq):
    idxs = [vocab[w] for w in seq.split()]
    return idxs + [pad]*(max_len - len(idxs))

inputs = torch.tensor([pad_seq(s) for s,_ in sent_pairs])      # Encoder input
targets = torch.tensor([pad_seq(t) for _,t in sent_pairs])     # Decoder target

print("inputs:\n", inputs)
print("targets:\n", targets)

inputs:
 tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]])
targets:
 tensor([[ 4,  3,  2,  1],
        [ 8,  7,  6,  5],
        [12, 11, 10,  9],
        [16, 15, 14, 13]])


**Train**

### Exercise 2:

In [ ]:
# TODO: define the training
model = __fill__
criterion = __fill__
opt = __fill__

for epoch in range(300):
    opt.zero_grad()

    # construct decoder input：<bos> we use the location of pad index 0 as initial（or create new <bos>）
    bos = torch.full((targets.size(0),1), pad)  # <bos>
    tgt_in = torch.cat([bos, targets[:, :-1]], dim=1)

    out = model(inputs, tgt_in)   # transformer forward
    loss = criterion(out.reshape(-1, vocab_size + 1), targets.reshape(-1))
    loss.backward()
    opt.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss={loss.item():.4f}")

Epoch 0, Loss=2.8952
Epoch 200, Loss=0.0015
Epoch 400, Loss=0.0005
Epoch 600, Loss=0.0003
Epoch 800, Loss=0.0002


**Inference**

In [ ]:
# TODO: test if it's correct
def decode(model, sentence):
    model.eval()
    src = torch.tensor([pad_seq(sentence)])
    tgt = torch.full((1,1), pad)

    for _ in range(max_len):
        out = model(src, tgt)
        next_word = out[:,-1].argmax(-1, keepdim=True)
        tgt = torch.cat([tgt, next_word], dim=1)

    pred_idxs = tgt[0,1:].tolist()
    pred_words = [inv_vocab[i] for i in pred_idxs if i != pad]
    return " ".join(pred_words)

for s,_ in sent_pairs:
    print(f"{s}  →  {decode(model, s)}")

In [ ]:
# Expected output

I love deep learning  →  learning deep love I
Transformers are so powerful  →  powerful so are Transformers
Attention is a magic  →  magic a is Attention
Neural networks learn patterns  →  patterns learn networks Neural
